# Keras nets

In [3]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.python.keras import models, layers, Input, Sequential
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.engine.training import Model
from tensorflow.python.keras.layers import Conv2D, Flatten, Activation, Dense, BatchNormalization,\
                                        Add, MaxPooling2D, Dropout, MaxPool2D, ZeroPadding2D
from tensorflow.python.keras.regularizers import l2

### normal CNN

In [4]:
def create_model_cnn(input_shape, n_classes):
    model = Sequential()
    
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu", input_shape=input_shape))
    model.add(Conv2D(filters=128, kernel_size=(5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(1024, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dense(n_classes, activation="softmax"))

    model.compile(loss="categorical_crossentropy", 
                  optimizer="adam", 
                  metrics=["accuracy"])
    
    return model

### deep CNN

In [5]:
def create_model_deep_cnn(input_shape=(16, 16, 1), n_classes=11):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPool2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(n_classes, activation='softmax'))

    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss=tf.losses.softmax_cross_entropy,
                  metrics=['accuracy'])
   
    return model

### ResNet

In [6]:
def resnet_block(x, size, kernel_size=(5, 5)):
    y = BatchNormalization(momentum=0.9)(x)
    y = Activation('relu')(y)
    y = Conv2D(size, kernel_size=kernel_size, padding='same')(y)
    y = BatchNormalization(momentum=0.9)(y)
    y = Activation('relu')(y)
    y = Conv2D(size, kernel_size=kernel_size, padding='same')(y)
    return Add()([x, y])

def create_model_resnet(input_shape, n_classes, heads_number=1):
    inputs = Input(shape=input_shape)
    
    layer = Conv2D(32, kernel_size=(5, 5), strides=(2, 2), activation='relu')(inputs)
    layer = resnet_block(layer, 32)
    layer = resnet_block(layer, 32)
    
    layer = Conv2D(64, kernel_size=(2, 2), strides=(2, 2), activation='relu')(layer)
    layer = resnet_block(layer, 64)
    layer = resnet_block(layer, 64)
    
    layer = Conv2D(128, kernel_size=(2, 2), strides=(2, 2), activation='relu')(layer)
    layer = resnet_block(layer, 128)
    layer = resnet_block(layer, 128)
    
    layer = Conv2D(256, kernel_size=(2, 2), strides=(2, 2), activation='relu')(layer)
    layer = resnet_block(layer, 256)
    layer = resnet_block(layer, 256)
    layer = BatchNormalization(momentum=0.9)(layer)
    
    layer = Flatten()(layer)
    
    layer = Dense(2048, activation='relu')(layer)
    outputs = [Dense(n_classes)(layer) for _ in range(heads_number)]
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss=[tf.losses.softmax_cross_entropy for _ in range(heads_number)], 
                  metrics=['accuracy'])
    
    return model

### AlexNet

In [7]:
def create_model_alexnet(input_shape=(32, 32, 3), n_classes=10, l2_reg=0.):
    model = Sequential()

    model.add(Conv2D(32, (5, 5), input_shape=input_shape, padding='same', kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2048))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(n_classes))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    model.compile(optimizer=tf.train.AdamOptimizer(), 
                  loss=tf.losses.softmax_cross_entropy,
                  metrics=['accuracy'])

    return model

# Data Preparation

In [9]:
!git clone https://github.com/yuridadt/play_with_keras.git

In [10]:
from IPython.display import clear_output

import os
import cv2
from typing import Tuple

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

IMG_WIDTH, IMG_HEIGHT = 48, 48
DATA_PATH = './play_with_keras/data'

In [11]:
def read_image(path: np.ndarray) -> np.ndarray:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    return img

In [12]:
def prepare_data(data_path: str = DATA_PATH) -> Tuple[np.ndarray, np.ndarray, np.array, np.array]:
    train_imgs, test_imgs = [], []
    y_train, y_test = [], []

    for dataset in ("train", "test"):
        for label, label_name in enumerate(("class 1", "class 2")):
            print(dataset, '-', label_name)
            for img in tqdm_notebook(os.listdir(f"{DATA_PATH}/isic_{dataset}/{label_name}")):
                image = read_image(f"{DATA_PATH}/isic_{dataset}/{label_name}/{img}")
                
                if dataset == "train":
                    train_imgs.append(image)
                    y_train.append(label)
                else:
                    test_imgs.append(image)
                    y_test.append(label)
            clear_output(wait=True)
            
    X_train = np.array(train_imgs)
    X_test = np.array(test_imgs)

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    return X_train, X_test, y_train, y_test

In [13]:
X_train, X_test, y_train, y_test = prepare_data()

test - class 2


# Visualisations

In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [15]:
def plot_samples(X: np.ndarray = X_train, y: np.array = y_train):
    titles = [''] * 10
    titles[2] = "Melanoma"
    titles[-3] = "No melanoma"
    ramdom_img = lambda X: X[np.random.randint(X.shape[0])]
    fig = make_subplots(rows=2, cols=5, start_cell="top-left", subplot_titles=titles)
    
    for col in range(1, 6):
        fig.add_trace(go.Image(z=ramdom_img(X[np.where(y == 1)])), row=1, col=col)
                      
    for col in range(1, 6):
        fig.add_trace(go.Image(z=ramdom_img(X[np.where(y == 0)])), row=2, col=col)
            
    fig.update_layout(title_text="Skin examples")
    fig.show()

In [16]:
def plot_history(history):
    fig = make_subplots(rows=1, cols=2, start_cell="top-left", subplot_titles=("Loss", "Acc"))
    
    x = [*range(1, len(history.history['loss']) + 1)]
    
    fig.add_trace(go.Scatter(x=x, y=history.history['loss'], name="Train loss"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=history.history['val_loss'], name="Val loss"), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=x, y=history.history['acc'], name="Train accuracy"), row=1, col=2)
    fig.add_trace(go.Scatter(x=x, y=history.history['val_acc'], name="Val accuracy"), row=1, col=2)
    
    fig.update_layout(title_text="Validation curves")
    fig.show()

In [22]:
plot_samples()

# Train

In [12]:
def shuffle_data(X: np.ndarray, y: np.array) -> Tuple[np.ndarray, np.array]:
    assert X.shape[0] == y.shape[0]
    
    shuffled_indices = np.arange(X.shape[0])
    np.random.shuffle(shuffled_indices)
    X, y = X[shuffled_indices], y[shuffled_indices]
    return X, y

In [13]:
X_train_norm = X_train / 255.0
X_test_norm = X_test / 255.0

y_train_onehot = to_categorical(y_train, 2)
y_test_onehot = to_categorical(y_test, 2)

In [14]:
X_train_norm, y_train_onehot = shuffle_data(X_train_norm, y_train_onehot)
X_test_norm, y_test_onehot = shuffle_data(X_test_norm, y_test_onehot)

In [19]:
model = create_model_resnet(input_shape=(48, 48, 3), n_classes=2)

In [20]:
history = model.fit(X_train_norm, y_train_onehot, 
                    validation_data=(X_test_norm, y_test_onehot), 
                    batch_size=75, epochs=5)

Train on 1255 samples, validate on 400 samples
Epoch 1/5
1255/1255 [==============================] - 25s 20ms/sample - loss: 1.3725 - acc: 0.5976 - val_loss: 0.6235 - val_acc: 0.6050
Epoch 2/5
1255/1255 [==============================] - 23s 18ms/sample - loss: 0.6063 - acc: 0.6661 - val_loss: 0.6227 - val_acc: 0.6550
Epoch 3/5
1255/1255 [==============================] - 23s 19ms/sample - loss: 0.5050 - acc: 0.7386 - val_loss: 0.3642 - val_acc: 0.8475
Epoch 4/5
1255/1255 [==============================] - 25s 20ms/sample - loss: 0.4141 - acc: 0.8104 - val_loss: 0.3155 - val_acc: 0.8500
Epoch 5/5
1255/1255 [==============================] - 23s 18ms/sample - loss: 0.4196 - acc: 0.8040 - val_loss: 0.4518 - val_acc: 0.7725


In [21]:
plot_history(history)